In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
dframe= pd.read_csv('C:/Users/Harsh/Desktop/npcl.5_id_data.csv')

In [3]:
id=dframe['location_id'].unique()

In [4]:
d_frame=dframe[['data.grid_reading_kwh','data.grid_kwh_unit','data.creation_time','data.status',
'data.meter_sr_no','data.error_reason','data.R_voltage','data.Y_volatge',
'data.B_voltage', 'data.r_current', 'data.y_current','data.b_current','site_id',
'location_id','data.cumm_pf','data.frequency']]

In [19]:
# combined_df['data.cumm_pf'].plot(figsize=(18,5))

checks

In [5]:
# Status "P"
combined_df=d_frame[d_frame['data.status']== "P"]

#no consumption
combined_df.loc[combined_df['data.error_reason'] == 'No Consumption', 'data.grid_kwh_unit'] = 0

# null values
if combined_df.isnull().sum().sum()>0:
    combined_df.dropna(inplace=True)

# special character
def special_char():
    special_character_pattern = r'[!@#$%^&*()]'
    rows_with_special_chars = combined_df[combined_df['location_id'].str.contains(special_character_pattern, regex=True)]
    return ( rows_with_special_chars)
if combined_df['location_id'].dtype != 'int64':
    special_char()

#  creation time relabelling 
# combined_df['data.creation_time']=pd.to_datetime(combined_df['data.creation_time'])

# integer conversion
combined_df['data.grid_reading_kwh']=combined_df['data.grid_reading_kwh'].round()

In [6]:
final_lst=[]
for each_id in id:
    df=combined_df[combined_df['location_id'] == each_id ]
    df['data.creation_time']=pd.to_datetime(df['data.creation_time'])
    df.set_index(['data.creation_time'],drop=True, inplace=True)
    
    # dfresample=df[['data.grid_reading_kwh']].resample(rule='1H').max()

    # df['units']=round((df['data.grid_reading_kwh']-df['data.grid_reading_kwh'].shift(1)),2)
    # df.dropna(inplace=True)
    # df = df[df['units'] >= 0]
    df_resample=df[['data.grid_reading_kwh']].resample(rule='1H').max()
    df_resample['data.grid_reading_kwh'].interpolate(inplace=True)
    df_resample['grid_reading']=round(df_resample['data.grid_reading_kwh'])
    df_resample['unit']=df_resample['grid_reading']-df_resample['grid_reading'].shift(1)
    df_resample = df_resample[df_resample['unit'] >= 0]
    
    

    final_lst.append(df_resample)

final_df = pd.concat(final_lst)

In [26]:
df=final_lst[2]